In [1]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Initialize the RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)  # Enable depth stream
pipeline.start(config)

# Create a tracker object
tracker = cv2.TrackerCSRT_create()

# Initialize variables for tracking
tracking = False
bbox = None
tracked_data = []

while True:
    # Wait for the next set of frames
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()  # Retrieve depth frame
    
    if color_frame and depth_frame:
        # Convert the color frame to a numpy array
        color_image = np.asanyarray(color_frame.get_data())
        
        if not tracking:
            # Display instructions
            cv2.putText(color_image, "Select object to track and press 's' to start tracking", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if tracking:
            # Update the tracker
            ok, bbox = tracker.update(color_image)
            
            if ok:
                # Tracking successful, draw bounding box
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(color_image, p1, p2, (0, 255, 0), 2)
                
                # Retrieve depth value at the center of the bounding box
                center_x = int((p1[0] + p2[0]) / 2)
                center_y = int((p1[1] + p2[1]) / 2)
                depth_value = depth_frame.get_distance(center_x, center_y)
                
                # Store the bounding box coordinates and depth value
                tracked_data.append(p1 + p2 + (depth_value,))
            else:
                # Tracking failed, display message
                cv2.putText(color_image, "Tracking failed", (20, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                tracking = False

        # Display the color image
        cv2.imshow('Object Tracking', color_image)
        
    # Check for key press
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):
        # Start tracking when 's' is pressed
        bbox = cv2.selectROI('Object Tracking', color_image, fromCenter=False, showCrosshair=True)
        tracker.init(color_image, bbox)
        tracking = True
    elif key == 27:
        # Exit program when 'ESC' is pressed
        break

# Stop the pipeline and close all OpenCV windows
pipeline.stop()
cv2.destroyAllWindows()

# Save the tracked data to a file
np.savetxt('tracked_data.txt', tracked_data)


In [2]:
tracked_data

[(398, 165, 509, 287, 0.581000030040741),
 (395, 163, 511, 290, 0.5820000171661377),
 (395, 164, 511, 291, 0.581000030040741),
 (397, 167, 508, 289, 0.5840000510215759),
 (395, 165, 509, 290, 0.5830000042915344),
 (396, 167, 507, 289, 0.5860000252723694),
 (395, 166, 509, 291, 0.5830000042915344),
 (394, 164, 510, 291, 0.5840000510215759),
 (395, 165, 509, 290, 0.5840000510215759),
 (394, 164, 510, 291, 0.5850000381469727),
 (396, 165, 510, 290, 0.581000030040741),
 (395, 166, 509, 291, 0.5830000042915344),
 (397, 169, 506, 289, 0.5840000510215759),
 (396, 171, 503, 288, 0.5890000462532043),
 (393, 171, 502, 291, 0.5940000414848328),
 (388, 168, 504, 295, 0.5940000414848328),
 (385, 168, 508, 303, 0.5950000286102295),
 (384, 168, 507, 303, 0.597000002861023),
 (384, 168, 507, 303, 0.597000002861023),
 (382, 168, 508, 305, 0.597000002861023),
 (383, 169, 509, 306, 0.5950000286102295),
 (383, 168, 509, 305, 0.5960000157356262),
 (383, 166, 509, 303, 0.5960000157356262),
 (384, 164, 512, 